## Lyman-$\alpha$ Polarization Signal From Reionization Fronts


Currently in this notes, we have 4 functions to code up: 

- **u_front**: calculate the ionizing front velocity
- **xH_interp**: do a 3D interpolation for the xH boxes
- **front_loc**: identify the location of xH=0.5 cells, which are regarded as the ionizing fronts
- **xH_grad**: get the xH gradient value at ionizing fronts

**Emily and Yuanyuan, pick one or more functions you'd like to build up or play with after reading the notes.** You could write your name after the function you choose above, e.g. u_front (Heyang), or just send a slack messege to let other teammates know. As you may have noticed there are so many pieces in this calculation and we may need to work on communicating more often to make sure we are on the same page and could work more collaborately and efficiently.

---
## General descrition
Polarized Lyman-$\alpha$ signal could be generated by ionizing fronts when Lyman-$\alpha$ photons were scatterer by HI. Therefore, the power spectrum of Lyman-$\alpha$ polarization intensity mapping signal at the Epoch of Reionization should have a feature scale which is comparable to the scale of reionization bubbles. As illustrated by Yuanyuan and Emily's first paper, this signal depends on local hydrogen density, the velocity, temperature and orientation of front plane, which could shed light not only on the process of reionization, but also the geometrical information of reionization. In this project we want to extend the results of single-plane parallel inoization front to have a more realistic estimate of signal from reionization fronts. To achive this, we first run 21cmFAST simulations with reionization implemented, then we extract quantities from simulation boxes for the sake of our calculations. Lastly, with the polarization intensity distribution from Yuanyuan and Emily's paper and quantites from 21cmFAST, we could finish the aimed calculation.

### Box file loading

The output boxes of 21cmFAST v2 are binary files (if you run simulations with 21cmFAST v3 then the stored format is different from here), so we use the 'load_binary_data' function to read box data. The dimensions of our trial simulation boxes are (200,200,200), while the box file has an array of length $200^3$. The array could be reshaped as shown below.

In [8]:
import numpy as np
import sys
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

def load_binary_data(filename, dtype=np.float32): 
     """ 
     We assume that the data was written 
     with write_binary_data() (little endian). 
     """ 
     f = open(filename, "rb") 
     data = f.read() 
     f.close() 
     _data = np.fromstring(data, dtype) 
     if sys.byteorder == 'big':
       _data = _data.byteswap()
     return _data 
    
box = load_binary_data('../data/z_first_ionization_z005.90_HIIfilter1_RHIImax50_200_300Mpc')
box.shape=(200,200,200)
# box


<ipython-input-8-a393ec2f9a32>:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  _data = np.fromstring(data, dtype)


### Quantities

#### Velocity

As we discussed, the velocity of fronts can be calculated by

$$
\frac{n_H(1+f_{\rm He})u}{1-u/c}=\int^{\infty}_{I_H/h}d\nu\frac{F_{\nu}^{inc}}{h\nu} = \frac{\Gamma_{\rm HI}\,(ionized\, side)}{\overline{\sigma}_{\rm HI}}
$$
where $n_H$ is the hydrogen density, $f_{\rm He}$ is the helium fraction, u is the velocity of ionizing front, c is the speed of light, $\Gamma_{\rm HI}$ is the photoionization rate, $\overline{\sigma}_{\rm HI}$ is the mean thompson scattering cross section. 

- $n_{\rm H}$

21cmFAST doesn't save number density directly, but it does store the matter overdensity data in files with name starts with "updated_smoothed_deltax". So we could calculate the hydrogen number density by

$$
n_{\rm H} = \overline{n}_{\rm H}(1+\delta_b)\approx\frac{\Omega_b(z) Y_H \rho_{\rm crit}}{m_{\rm H}}(1+\delta_m)
$$

where $\Omega_b$ is the baryon abundance at the redshift we investigate, $Y_H$ is the hydrogen fraction, $\rho_{\rm crit}$ is the critical denstiy, $m_{\rm H}$ is the hydrogen atom mass and $\delta_m$ is the matter overdensity we could extract from simulation box. 

- $\overline{\sigma}_{\rm H}$ 

Yuanyuan has developed the function to calculate $\overline{\sigma}_{\rm H}$.

**Define a function encoding the calculation of front velocity in the cell below**



In [10]:
def u_front(gamma_file,delta_file,coordinates):
    '''
    gamma_file : test file is '../data/Gamma12aveHII_z008.11_HIIfilter1_RHIImax50_200_300Mpc'
    delta_file : test file is '../data/updated_smoothed_deltax_z008.11_200_300Mpc'
    coordinates : point coordinates in our box
    '''
    Gamma = load_binary_data(gamma_file)
    delta_m = load_binary_data(delta_file)
    
    gamma_HI = 
    nH = 
    
    u = 
    
    return u
    
    

SyntaxError: invalid syntax (<ipython-input-10-d01cc02f1121>, line 10)

#### Fronts geometry

One of the key pieces we need to extract from 21cmFAST boxes is the location and orientation of ionizing fronts. Files starts with "xH_nohalos" from 21cmFAST have the ionized hydrogen fraction data for each cell. The original box is $200\times200\times200$, as we discussed we decided to define the ionizing fronts as 3D plane connedted by 
points where $x_{\rm H}=0.5$. This could be done by interpolating the original box and get a list of $x_{\rm H}=0.5$ points.

**Write a function that reads the ionized HI fraction box and does 3D interpolation for it in the cell below. The return obeject of this function should be a function xH(x,y,z) that could return the xH value at a given coordinate.**

In [ ]:
from scipy.interpolate import NearestNDInterpolator
# read scipy document to understand how to implement this interpolation method
# you could also try other approaches to do this interpolation

def xH_interp(filename):
'''
filename :  test file is '../data/'xH_nohalos_z008.11_nf0.543148_eff25.0_effPLindex0_HIIfilter1_Mmin5.0e+08_RHIImax50_200_300Mpc''
'''
    xH = load_binary_data(filename)
    xH.shape = (200,200,200)
    
    
    xH_func = 
    
    return xH_func

**Moreover, try to use the above function to identify cells where $x_{\rm H}=0.5$ in a $1600\times1600\times1600$ box.** (We choose 1600 here to initially develop our codes, while resolution convergence tests are necessary after we complete building the calculation method.) Note that it's impractical to complete the calculation for the whole $1600\times1600\times1600$ box due to the computational memory limit, we may want to get these values layer by layer, i.e. calculate all the way through for a $1600^2\times8$ layer once, save values we need and then move forward to the next layer.

In [ ]:
xH_func = xH_interp()

def front_loc(resolution, n_layer, layer_order):
'''
In this function, at first we want an array representing all coordinates we need. 
Then we loop over these coordinates and flag those satisfy the front boundary condition.

resolution : 1600 for current case
n_layer : 200
'''    
    # 1600^2*8 array
    coordinates = 
    front_loc = []
    
    for points in coordinates:
        
        if : # front boundary condition here
            front_loc.append(points)
    
    return front_loc

The intensity signal would be an integral over the ionizing fronts area $I \propto \int d\vec{A}\, P(\vec{n},u,n_{\rm H},T)$, where P is the polartized intensity, $\vec{n}$ is the normal vector of the front plane. Recall that we could find $x_{\rm H}=0.5$ cells through the function defined above, it's still tricky for us to connect these points and define a "plane" from them. To tackle this problem, we could transform the area integral to a volume integral

$$
\int_{\rm fronts} d\vec{A} = \int \frac{d V}{\rm thickness} = \int \frac{d V}{\Delta x_{\rm HI}/|\vec{\triangledown}x_{\rm HI}|}=\int_{\rm box} {d V}\frac{1}{\Delta x_{\rm HI}}|\vec{\triangledown}x_{\rm HI}|
$$

We substitute $\frac{1}{\Delta x_{\rm HI}}$ by delta function $\delta(x_{\rm HI}-0.5)$ in the limit $\Delta x_{\rm HI} \rightarrow 0$ at the boundary of ionizing fronts. So the integral becomes

$$
I\propto \int_{box} {d V}\, \delta(x_{\rm HI}-0.5)\, |\vec{\triangledown}x_{\rm HI}|P(\vec{n},u,n_{\rm H},T)=\sum V_{\rm cell}\,|\vec{\triangledown}x_{\rm HI}|_{x_{\rm H}=0.5}\times P(\vec{n},u,n_{\rm H},T)\mid_{x_{\rm H}=0.5}
$$

Now the integral becomes a sum of quantites at the $x_{\rm H}=0.5$ cells, **we successfully avoid the difficulty of defining the fronts plane!** As we already have boundary location by front_loc, one more thing we need is $|\vec{\triangledown}x_{\rm HI}|$ at these cells. Note that the front orientation information also lies in the gradient of $x_{\rm HI}$, so it's a crucial step to have $\vec{\triangledown}x_{\rm HI}$ in our calculation.

**Write a function to get the gradient of $x_{\rm HI}$ at the cell below.** Recall that in front_loc, one layer has $1600^2\times8$ cells, but we should need 3 layers in order to get the gradient for each layer, i.e. $1600^2\times24$ cells.

In [ ]:
def xH_grad():
''' 
The inputs for this function could be layers location
output should be an array of the xH gradient value and coordinates corresponding to xH=0.5 cells
'''
    # get the 1600^24 cell coordinates here
    coordinates = 
    
    # get the xH values for these cells using xH_interp
    xHs = 
    
    # get the xH=0.5 cell locations here using front_loc
    front_locs = 
    
    # get the gradients for the identified cells
    xH_grad_func = 
    
    
    return xH_grad, front_locs